In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../'

/Users/laurentperrinet/quantic/science/HomeHots/HOTS_clone_laurent


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import time

from HOTS.ToolsMonitor import GenerateActivationMap, DisplayActivationMap
from HOTS.Event import Event, SimpleAlphabet, LoadNMNIST
from HOTS.STS import STS
from HOTS.Layer import ClusteringLayer
from HOTS.ToolsMonitor import (
    DisplayImage,
    DisplaySurface3D,
    DisplaySurface2D,
    DisplayConvergence,
    DisplayLayer,
)
from HOTS.Tools import SaveObject, LoadObject
from HOTS.Classifier import Classifier
from HOTS.Network import Network
from HOTS.KmeansLagorce import KmeansLagorce
from HOTS.KmeansMaro import KmeansMaro
from HOTS.conv2eve import conv2eve

tau = 9e-4 # -> tau=1ms, si on prend 10 ms on est à 1s pour la dernière couche et les vidéos font 0.3s en moyenne
R = 2
filthr = 2
nbkNN = 3
algo = 'lagorce'
decay = 'exponential'
hom = True
krnlinit='rdn'
nb_cluster = [4, 8, 16]
ImageSize = (34, 34)
DataPath = 'Data/testsetnmnist.p'

NbClusteringData = 15
NbTrainingData = 40
NbTestingData = 40

event_tr, event_te, event_cl, label_tr, label_te = LoadNMNIST(
NbTrainingData, NbTestingData, NbClusteringData, Path=DataPath, OutOnePolarity=False, ListPolarities=None, verbose=0)

In [3]:
# with homeostasis
timestr = '20201028_xNsur4_'
fname = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'ms_'+algo+'_homeo.pkl'

ClusterLayer = LoadObject(fname)
DisplayLayer(ClusterLayer, maxpol=5, hisiz=2, yhis=0.3)
plt.savefig('notebooks/fig/layerzhomeo.png', format='png')

FileNotFoundError: [Errno 2] No such file or directory: 'Records/EXP_03_NMNIST/20201028_xNsur4__hots_0.9ms_lagorce_homeo.pkl'

In [ ]:
# without homeostasis
timestr = '20201021'
fname = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'ms_'+algo+'.pkl'

ClusterLayer, Classif0 = LoadObject(fname)
DisplayLayer(ClusterLayer, maxpol=5, hisiz=2, yhis=0.5)
plt.savefig('notebooks/fig/layerznohomeo.png', format='png')

In [ ]:
homrun = False
event0_o_tr, event0_olz_tr = Net.RunNetwork2(event_tr, NbClusterList=ClusterLayer, homrun=homrun)
        ## Generating histogram for the test samples
event0_o_te, event0_olz_te = Net.RunNetwork2(event_te, NbClusterList=ClusterLayer, homrun=homrun)

In [ ]:
#For the first layer

Classif0 = Classifier(
            event0_olz_tr[0], event0_olz_te[0], TrainingLabel=label_tr, GroundTruth=label_te
        )
prediction, accuracy, method = Classif0.HistogramDistance(knn=nbkNN, to_print=True)

In [ ]:
#For the second layer

Classif0 = Classifier(
            event0_olz_tr[1], event0_olz_te[1], TrainingLabel=label_tr, GroundTruth=label_te
        )
prediction, accuracy, method = Classif0.HistogramDistance(knn=nbkNN, to_print=True)

In [ ]:
#For the second layer

Classif0 = Classifier(
            event0_olz_tr[2], event0_olz_te[2], TrainingLabel=label_tr, GroundTruth=label_te
        )
prediction, accuracy, method = Classif0.HistogramDistance(knn=nbkNN, to_print=True)

In [ ]:
np.sqrt(289)